# ALI Performance Tests on Blake

In [1]:
import datetime as dt
import glob
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import os

import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode

# Import scripts
from json2status import json2status
from json2timeline import json2timeline
from utils import *
from models import cusum


# Extract file names
files = glob.glob('ctest-*')

In [2]:
# Enable offline plot export
init_notebook_mode(connected=True)

## Performance Timelines

In [12]:
# %matplotlib inline
# #plt.rc('text', usetex=True)
# plt.rc('font', family='serif', size=24)
# plt.rcParams['figure.figsize'] = (12, 6)

# Cases to plot:
cases = ('ant-2-20km_ml_ls','ant-2-20km_mu_ls','ant-2-20km_mu_dls')
nproc = 384
names = ('Total Time',
#          'Total Fill Time',
         'NOX Total Preconditioner Construction',
         'NOX Total Linear Solve')
timers = ('Albany Total Time:',
#           'Albany: Total Fill Time:',
          'NOX Total Preconditioner Construction:',
          'NOX Total Linear Solve:')

data = {}
for case in cases:
    data[case] = {}
    for name, timer in zip(names, timers):
        dates, wtimes = json2timeline(files, case, nproc, timer, False)
        fdates = [dt.datetime.strptime(str(d),'%Y%m%d').date() for d in dates]

        # Run algorithm
        wtimes = np.array(wtimes)
        output, changePts = cusum(wtimes, threshold = 4.5, max_influence=3, verbose=0)
        mean, std = regimeTimeseries(wtimes, changePts)
        tempDict = dict(
            date=fdates,
            time=wtimes,
            mean=mean,
            upper=mean+2*std,
            lower=mean-2*std
        )
        data[case][name] = pd.DataFrame(tempDict)

In [15]:
lines = ['time', 'mean', 'upper', 'lower']
colors = ['blue', 'red', 'red', 'red']
modes = ['markers', 'lines', 'lines', 'lines']
dashes = ['solid', 'solid', 'dash', 'dash']

fig = go.Figure()
for line, color, mode, dash in zip(lines, colors, modes, dashes):
    for c in cases:
        fig.add_trace(go.Scatter(
            x=data[c][names[0]]['date'],
            y=data[c][names[0]][line],
            mode=mode,
            line = dict(color=color, dash=dash),
            name=names[0] if line==lines[0] else line,
            visible=True if c==cases[0] else False
        ))

caseOptions = [dict(
        args=['visible', [True if x==c else False for x in np.tile(cases, len(lines))]],
        label=c,
        method='restyle'
    ) for c in cases]
    
nameOptions = [dict(
        args=['y', [data[c][n][line] for line in lines for c in cases]],
        label=n,
        method='restyle'
    ) for n in names]

fig.update_layout(
    updatemenus=[
        go.layout.Updatemenu(
            buttons=list(caseOptions),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
        go.layout.Updatemenu(
            buttons=list(nameOptions),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.23,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
    ],
    xaxis_title='Simulation Date',
    yaxis_title='Wall-clock Time (s)'
)

iplot(fig)